In [18]:
!git clone https://github.com/sayakpaul/Sharpness-Aware-Minimization-TensorFlow

fatal: destination path 'Sharpness-Aware-Minimization-TensorFlow' already exists and is not an empty directory.


In [19]:
import sys
sys.path.append("Sharpness-Aware-Minimization-TensorFlow")

In [20]:
import tensorflow as tf
tf.random.set_seed(42)
print(tf.__version__)

2.4.1


In [21]:
try: # detect TPUs
    tpu = None
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)
print("hello")

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of accelerators:  1
hello


In [22]:
import matplotlib.pyplot as plt
import resnet_cifar10
import utils
import time

In [23]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(f"Training samples: {len(x_train)}")
print(f"Testing samples: {len(x_test)}")

Training samples: 50000
Testing samples: 10000


In [24]:
BATCH_SIZE = 128 * strategy.num_replicas_in_sync
print(f"Batch size: {BATCH_SIZE}")
AUTO = tf.data.AUTOTUNE

def scale(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.cast(label, tf.int32)
    return image, label

def augment(image,label):
    image = tf.image.resize_with_crop_or_pad(image, 40, 40) # Add 8 pixels of padding
    image = tf.image.random_crop(image, size=[32, 32, 3]) # Random crop back to 32x32
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    image = tf.clip_by_value(image, 0., 1.)

    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = (
    train_ds
    .shuffle(1024)
    .map(scale, num_parallel_calls=AUTO)
    .map(augment, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = (
    test_ds
    .map(scale, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

Batch size: 128


In [25]:
import tensorflow as tf
tf.config.run_functions_eagerly(False)

class SAMModel(tf.keras.Model):
    def __init__(self, resnet_model, rho=0.05):
        """
        p, q = 2 for optimal results as suggested in the paper
        (Section 2)
        """
        super(SAMModel, self).__init__()
        self.resnet_model = resnet_model
        self.rho = rho

    def train_step(self, data):
        (images, labels) = data
        e_ws = []
        with tf.GradientTape() as tape:
            predictions = self.resnet_model(images)
            loss = self.compiled_loss(labels, predictions)
        trainable_params = self.resnet_model.trainable_variables
        gradients = tape.gradient(loss, trainable_params)
        grad_norm = self._grad_norm(gradients)
        scale = self.rho / (grad_norm + 1e-12)

        for (grad, param) in zip(gradients, trainable_params):
            e_w = grad * scale
            param.assign_add(e_w)
            e_ws.append(e_w)

        with tf.GradientTape() as tape:
            predictions = self.resnet_model(images)
            loss = self.compiled_loss(labels, predictions)    
        
        sam_gradients = tape.gradient(loss, trainable_params)
        for (param, e_w) in zip(trainable_params, e_ws):
            param.assign_sub(e_w)
        
        self.optimizer.apply_gradients(
            zip(sam_gradients, trainable_params))
        
        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        (images, labels) = data
        predictions = self.resnet_model(images, training=False)
        loss = self.compiled_loss(labels, predictions)
        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

    def _grad_norm(self, gradients):
        norm = tf.norm(
            tf.stack([
                tf.norm(grad) for grad in gradients if grad is not None
            ])
        )
        return norm

In [26]:
train_callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5,
        patience=3, verbose=1
    )
]

In [27]:
with strategy.scope():
    model = SAMModel(utils.get_training_model())
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
print(f"Total learnable parameters: {model.resnet_model.count_params()/1e6} M")

Total learnable parameters: 0.575114 M


In [28]:
start = time.time()
history = model.fit(train_ds,
                   validation_data=test_ds,
                   callbacks=train_callbacks,
                   epochs=100)
print(f"Total training time: {(time.time() - start)/60.} minutes")

Epoch 1/100
391/391 [==============================] - 19s 35ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/100
391/391 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.0999 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/100
391/391 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.0993 - val_loss: nan - val_accuracy: 0.1000

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/100
391/391 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/100
391/391 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.0998 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/100
391/391 [==============================] - 12s 30ms/step - loss: nan - accuracy: 0.1001 - val_loss: nan - val_accuracy: 0.1000

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoc

TypeError: object of type 'NoneType' has no len()

In [16]:
utils.plot_history(history)

NameError: name 'history' is not defined

In [17]:
with strategy.scope():
    model = utils.get_training_model()

model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

start = time.time()
history = model.fit(train_ds,
                   validation_data=test_ds,
                   callbacks=train_callbacks,
                   epochs=200) # 200 eppochs since SAM takes two backprop steps for an update
print(f"Total training time: {(time.time() - start)/60.} minutes")

Epoch 1/200
391/391 [==============================] - 14s 22ms/step - loss: 2.3027 - accuracy: 0.0944 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/200
391/391 [==============================] - 7s 19ms/step - loss: 2.3027 - accuracy: 0.0932 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/200
391/391 [==============================] - 8s 19ms/step - loss: 2.3027 - accuracy: 0.0964 - val_loss: nan - val_accuracy: 0.1000

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/200
391/391 [==============================] - 7s 19ms/step - loss: 2.3027 - accuracy: 0.0966 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/200
391/391 [==============================] - 7s 19ms/step - loss: 2.3027 - accuracy: 0.0954 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/200
391/391 [==============================] - 8s 20ms/step - loss: 2.3027 - accuracy: 0.0952 - val_loss: nan - val_accuracy: 0.1000

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.000250000011

TypeError: object of type 'NoneType' has no len()

In [ ]:
utils.plot_history(history)